Bank Nifty short Straddle

Selling 1 CE ATM & 1 PE ATM option at 9:20 and exit at 15:10 with a stoploss of 20% each

In [1]:
#importing all liabraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta,TH

In [63]:
#fething all data
path=pd.DataFrame(glob('E:/sample_nfo_2019-20_data/*'),columns=['location'])
path['datadate']=path['location'].apply(lambda x: x.split('_')[-1].split('.')[0])
path['datadate']=path['datadate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
path=path.sort_values('datadate')
path.reset_index(drop=True,inplace=True)

In [64]:
trade_log=pd.DataFrame(columns=['Entry_date_time', 'Future_price', 'ATM', 'Days_to_expiry',
          'CE_symbol','CE_entry_price','CE_exit_price','CE_exit_datetime','CE_pnl',
          'PE_symbol','PE_entry_price','PE_exit_price','PE_exit_datetime','PE_pnl', 'Total_pnl'])

In [65]:
for index, row in path.iterrows():
    try:
        print(index)
        data=pd.read_pickle(row['location'])
    
        entry_datetime=datetime.datetime.combine(row['datadate'].date(),datetime.time(9,20))
        exit_dateitme=datetime.datetime.combine(row['datadate'].date(),datetime.time(15,10))

        # bnf future data & ATM
        data['expiry_type']=np.where((data['instrument_type']=='FUT'),(data['ticker'].apply(lambda x:x.split('-')[-1].split('.')[0])),"")
        current_month_exp= 'I' # I-current month expiry, II-next month expiry, III- far month expiry
        instrument='BANKNIFTY'
        future_data=data[(data['instrument_type']=='FUT') & (data['instrument_name']==instrument) & (data['expiry_type']==current_month_exp)]
        future_data.reset_index(drop=True,inplace=True)
        atm=future_data[future_data['datetime']==entry_datetime]['open'].iloc[0]
        base_price=100
        atm=base_price*round(atm/base_price)

        # CE data & PE data
        nearest_expiry=row['datadate'].date()+relativedelta(weekday=TH)
        ce_data=data[(data['instrument_type']=='CE') & (data['instrument_name']==instrument) & (data['strike_price']==atm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        ce_data.reset_index(drop=True,inplace=True)
        pe_data=data[(data['instrument_type']=='PE') & (data['instrument_name']==instrument) & (data['strike_price']==atm) & ((data['expiry_date']==nearest_expiry)|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=1))|(data['expiry_date']==nearest_expiry-datetime.timedelta(days=2)))]
        pe_data.reset_index(drop=True,inplace=True)

        #symbol & entry price
        ce_symbol=ce_data['ticker'].iloc[0]
        pe_symbol=pe_data['ticker'].iloc[0]
        future_price= future_data[future_data['datetime']==entry_datetime]['open'].iloc[0]
        ce_entry_price= ce_data[ce_data['datetime']==entry_datetime]['close'].iloc[0]
        pe_entry_price=pe_data[pe_data['datetime']==entry_datetime]['close'].iloc[0]

        #test data
        future_data=future_data[['datetime','close']].set_index(['datetime'])
        ce_data=ce_data[['datetime','close']].set_index(['datetime'])
        pe_data=pe_data[['datetime','close']].set_index(['datetime'])
        intraday_data=pd.concat([future_data,ce_data,pe_data],axis=1)
        intraday_data.columns=['future_close','ce_close','pe_close']
        intraday_data=intraday_data.ffill()
        intraday_data.reset_index(inplace=True)
        entry_datetime_index=intraday_data[intraday_data['datetime']==entry_datetime].index[0]
        exit_datetime_index=intraday_data[intraday_data['datetime']==exit_dateitme].index[0]
        intraday_data=intraday_data[entry_datetime_index:exit_datetime_index+1]
        intraday_data['ce_pnl']=0
        intraday_data['pe_pnl']=0
        intraday_data.reset_index(drop=True,inplace=True)

        #stoploss
        stop_loss= 20/100
        ce_stop_loss=ce_entry_price+ce_entry_price*stop_loss
        pe_stop_loss=pe_entry_price+pe_entry_price*stop_loss

        #backtest
        ce_stop_loss_counter=0
        pe_stop_loss_counter=0
        ce_exit_datetime=''
        pe_exit_datetime=''
        ce_exit_price=0
        pe_exit_price=0
        ce_pnl=0
        pe_pnl=0
        pnl=0

        for index, row in intraday_data.iterrows():
            ce_ltp=row['ce_close']
            pe_ltp=row['pe_close']
            #print(f"{row['datetime']}::{ce_ltp}::{pe_ltp}")
            #Exit crateria
            #1. CE & PE didn't hit SL & both reach exit time limit 15:10
            if (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0) & (row['datetime']==exit_dateitme):
                ce_pnl=ce_entry_price-ce_ltp
                pe_pnl=pe_entry_price-pe_ltp
                ce_stop_loss_counter=1
                pe_stop_loss_counter=1
                ce_exit_datetime=row['datetime']
                pe_exit_datetime=row['datetime']
                ce_exit_price=ce_ltp
                pe_exit_price=pe_ltp
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('CE & PE did not hit SL, both exit at 15:10')
                pnl=ce_pnl+pe_pnl
                break 
            #2. CE is now hit SL, none were hit SL till now
            elif (ce_ltp>=ce_stop_loss) & (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0):
                ce_pnl=ce_entry_price-ce_stop_loss
                pe_pnl=pe_entry_price-pe_ltp
                ce_stop_loss_counter=1
                ce_exit_datetime=row['datetime']
                ce_exit_price=ce_stop_loss
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('CE SL hit')
                pnl=ce_pnl+pe_pnl

            #3. PE is now hit SL, none were hit SL till now
            elif (pe_ltp>=pe_stop_loss) & (ce_stop_loss_counter==0) & (pe_stop_loss_counter==0):
                ce_pnl=ce_entry_price-ce_ltp
                pe_pnl=pe_entry_price-pe_stop_loss
                pe_stop_loss_counter=1
                pe_exit_datetime=row['datetime']
                pe_exit_price=pe_stop_loss
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                print('PE SL hit')
                pnl=ce_pnl+pe_pnl
            #4. CE was hit SL, (PE is either hit SL or reach exit time limit 15:10)
            elif (ce_stop_loss_counter==1) & (pe_stop_loss_counter==0):
                if (pe_ltp>=pe_stop_loss) & (row['datetime']<exit_dateitme):
                    pe_pnl=pe_entry_price-pe_stop_loss
                    pe_stop_loss_counter=1
                    pe_exit_datetime=row['datetime']
                    pe_exit_price=pe_stop_loss
                    intraday_data.loc[index,'ce_pnl']=ce_pnl
                    intraday_data.loc[index,'pe_pnl']=pe_pnl
                    print('PE SL hit')
                    pnl=ce_pnl+pe_pnl
                    break
                elif (row['datetime']==exit_dateitme):
                    pe_pnl=pe_entry_price-pe_ltp
                    pe_stop_loss_counter=1
                    pe_exit_datetime=row['datetime']
                    pe_exit_price=pe_ltp
                    intraday_data.loc[index,'ce_pnl']=ce_pnl
                    intraday_data.loc[index,'pe_pnl']=pe_pnl
                    print('PE exit at 15:10')
                    pnl=ce_pnl+pe_pnl
                    break     
            #5. PE was hit SL, (CE is either hit SL or reach exit time limit 15:10)
            elif (pe_stop_loss_counter==1) & (ce_stop_loss_counter==0):
                if (ce_ltp>=ce_stop_loss) & (row['datetime']<exit_dateitme):
                    ce_pnl=ce_entry_price-ce_stop_loss
                    ce_stop_loss_counter=1
                    ce_exit_datetime=row['datetime']
                    ce_exit_price=ce_stop_loss
                    intraday_data.loc[index,'ce_pnl']=ce_pnl
                    intraday_data.loc[index,'pe_pnl']=pe_pnl
                    print('CE SL hit')
                    pnl=ce_pnl+pe_pnl
                    break
                elif (row['datetime']==exit_dateitme):
                    ce_pnl=ce_entry_price-ce_ltp
                    ce_stop_loss_counter=1
                    ce_exit_datetime=row['datetime']
                    ce_exit_price=ce_ltp
                    intraday_data.loc[index,'ce_pnl']=ce_pnl
                    intraday_data.loc[index,'pe_pnl']=pe_pnl
                    print('CE exit at 15:10')
                    pnl=ce_pnl+pe_pnl
                    break
            #6. update pnl in normal time
            elif ((ce_stop_loss_counter==0 & pe_stop_loss_counter==0) | (ce_stop_loss_counter==1 & pe_stop_loss_counter==0) | (ce_stop_loss_counter==0 & pe_stop_loss_counter==1) | (ce_stop_loss_counter==1 & pe_stop_loss_counter==1)) or (row['datetime']<exit_dateitme):
                ce_pnl=ce_entry_price-ce_ltp
                pe_pnl=pe_entry_price-pe_ltp
                intraday_data.loc[index,'ce_pnl']=ce_pnl
                intraday_data.loc[index,'pe_pnl']=pe_pnl
                pnl=ce_pnl+pe_pnl



        trade_log=trade_log.append({'Entry_date_time': entry_datetime,
                                    'Future_price': future_price,
                                    'ATM': atm, 
                                    'Days_to_expiry': (nearest_expiry-entry_datetime.date()).days,
                                    'CE_symbol': ce_symbol,
                                    'CE_entry_price':ce_entry_price,
                                    'CE_exit_price': ce_exit_price,
                                    'CE_exit_datetime': ce_exit_datetime,
                                    'CE_pnl': ce_pnl,
                                    'PE_symbol': pe_symbol,
                                    'PE_entry_price': pe_entry_price,
                                    'PE_exit_price': pe_exit_price,
                                    'PE_exit_datetime': pe_exit_datetime,
                                    'PE_pnl':pe_pnl, 
                                    'Total_pnl': pnl},ignore_index=True)
    except Exception as e:
        print(e)
        print(row['location'])

0
PE SL hit
CE SL hit
1
PE SL hit
CE SL hit
2
CE SL hit
PE SL hit
3
PE SL hit
CE SL hit
4
PE SL hit
CE exit at 15:10
5
CE SL hit
PE exit at 15:10
6
PE SL hit
CE exit at 15:10
7
PE SL hit
CE exit at 15:10
8
PE SL hit
CE exit at 15:10
9
CE & PE did not hit SL, both exit at 15:10
10
CE SL hit
PE exit at 15:10
11
CE SL hit
PE exit at 15:10
12
PE SL hit
CE exit at 15:10
13
PE SL hit
CE exit at 15:10
14
CE SL hit
PE exit at 15:10
15
CE & PE did not hit SL, both exit at 15:10
16
PE SL hit
CE exit at 15:10
17
PE SL hit
CE SL hit
18
PE SL hit
CE exit at 15:10
19
PE SL hit
CE exit at 15:10


In [66]:
trade_log

,Entry_date_time,Future_price,ATM,Days_to_expiry,CE_symbol,CE_entry_price,CE_exit_price,CE_exit_datetime,CE_pnl,PE_symbol,PE_entry_price,PE_exit_price,PE_exit_datetime,PE_pnl,Total_pnl
0,2019-01-01 09:20:00,27260.00,27300,2,BANKNIFTY03JAN1927300CE.NFO,100.00,120.00,2019-01-01 14:31:00,-20.00,BANKNIFTY03JAN1927300PE.NFO,225.60,270.72,2019-01-01 09:54:00,-45.12,-65.12
1,2019-01-02 09:20:00,27427.00,27400,1,BANKNIFTY03JAN1927400CE.NFO,99.05,118.86,2019-01-02 10:21:00,-19.81,BANKNIFTY03JAN1927400PE.NFO,157.55,189.06,2019-01-02 09:35:00,-31.51,-51.32
2,2019-01-03 09:20:00,27241.10,27200,0,BANKNIFTY03JAN1927200CE.NFO,48.10,57.72,2019-01-03 09:38:00,-9.62,BANKNIFTY03JAN1927200PE.NFO,135.30,162.36,2019-01-03 11:36:00,-27.06,-36.68
3,2019-01-04 09:20:00,27177.30,27200,6,BANKNIFTY10JAN1927200CE.NFO,179.60,215.52,2019-01-04 11:41:00,-35.92,BANKNIFTY10JAN1927200PE.NFO,283.05,339.66,2019-01-04 09:45:00,-56.61,-92.53
4,2019-01-07 09:20:00,27508.20,27500,3,BANKNIFTY10JAN1927500CE.NFO,137.75,82.35,2019-01-07 15:10:00,55.40,BANKNIFTY10JAN1927500PE.NFO,204.00,244.80,2019-01-07 14:45:00,-40.80,14.60
5,2019-01-08 09:20:00,27327.65,27300,2,BANKNIFTY10JAN1927300CE.NFO,110.30,132.36,2019-01-08 09:49:00,-22.06,BANKNIFTY10JAN1927300PE.NFO,199.50,70.25,2019-01-08 15:10:00,129.25,107.19
6,2019-01-09 09:20:00,27709.45,27700,1,BANKNIFTY10JAN1927700CE.NFO,81.85,82.65,2019-01-09 15:10:00,-0.80,BANKNIFTY10JAN1927700PE.NFO,167.60,201.12,2019-01-09 10:05:00,-33.52,-34.32
7,2019-01-10 09:20:00,27702.45,27700,0,BANKNIFTY10JAN1927700CE.NFO,44.15,0.05,2019-01-10 15:10:00,44.10,BANKNIFTY10JAN1927700PE.NFO,120.15,144.18,2019-01-10 09:37:00,-24.03,20.07
8,2019-01-11 09:20:00,27665.50,27700,6,BANKNIFTY17JAN1927700CE.NFO,172.10,119.75,2019-01-11 15:10:00,52.35,BANKNIFTY17JAN1927700PE.NFO,259.00,310.80,2019-01-11 10:44:00,-51.80,0.55
9,2019-01-14 09:20:00,27397.90,27400,3,BANKNIFTY17JAN1927400CE.NFO,141.30,118.50,2019-01-14 15:10:00,22.80,BANKNIFTY17JAN1927400PE.NFO,221.90,216.00,2019-01-14 15:10:00,5.90,28.70
